**Compute Financial Ratios using WRDS/S&P Compustat**

In [1]:
# Import required libraries 
import pandas as pd
from finance_lib import calculate_ccc, get_wrds_data

In [2]:
query = """
            SELECT gvkey, tic, conm, datadate,  
                lt, at, lct, act, seq, dltt, dlc, ni, sale, cogs, oibdp, oiadp, invt, ppent, wcap, csho, prcc_f, re, rect, ap
            FROM comp.funda
            WHERE indfmt='INDL' 
            AND datafmt='STD'
            AND popsrc='D'
            AND consol='C'
            AND datadate >= '01/01/2010' 
            AND tic in ('AAPL')
            """
            
comp = get_wrds_data(username='frankhuettner', sql_query=query)

Loading library list...
Done


In [3]:
# Inspect data
comp.head()

,gvkey,tic,conm,datadate,lt,at,lct,act,seq,dltt,...,oibdp,oiadp,invt,ppent,wcap,csho,prcc_f,re,rect,ap
0,001690,AAPL,APPLE INC,2010-09-30,27392.0,75183.0,20722.0,41678.0,47791.0,0.0,...,19317.0,18385.0,1051.0,4768.0,20956.0,915.970,283.750,37123.0,9924.0,12015.0
1,001690,AAPL,APPLE INC,2011-09-30,39756.0,116371.0,27970.0,44988.0,76615.0,0.0,...,35612.0,33790.0,776.0,7777.0,17018.0,929.277,381.320,63284.0,11717.0,14632.0
2,001690,AAPL,APPLE INC,2012-09-30,57854.0,176064.0,38542.0,57653.0,118210.0,0.0,...,58446.0,55241.0,791.0,15452.0,19111.0,939.208,667.105,101788.0,18692.0,21175.0
3,001690,AAPL,APPLE INC,2013-09-30,83451.0,207000.0,43658.0,73286.0,123549.0,16960.0,...,55756.0,48999.0,1764.0,16597.0,29628.0,899.213,476.750,103785.0,20641.0,22367.0
4,001690,AAPL,APPLE INC,2014-09-30,120292.0,231839.0,63448.0,68531.0,111547.0,28987.0,...,60449.0,52503.0,2111.0,20624.0,5083.0,5866.161,100.750,88234.0,27219.0,30196.0


In [5]:
# Compute year from datadate
comp['datadate'] = pd.to_datetime(comp['datadate']) #convert datadate to date fmt
comp['year'] = comp['datadate'].dt.year #get year from datadate

In [6]:
## Define Financial Metrics

# Leverage Ratios
comp['debt'] = comp['dltt'] + comp['dlc'] # TOTAL DEBT = LONG TERM + SHORT TERM DEBT
comp['de_ratio'] = comp['debt'] / comp['seq'] # D/E RATIO = TOTAL DEBT / TOTAL SHAREHOLDER'S EQUITY
comp['da_ratio'] = comp['debt'] / comp['at'] # D/A RATIO = TOTAL DEBT / TOTAL ASSETS

# Liquidity Ratios 
comp['current_ratio'] = comp['act'] / comp['lct'] # CURRENT RATIO = CURRENT ASSETS / CURRENT LIABILITIES
comp['quick_ratio'] = (comp['act'] - comp['invt']) / comp['lct'] # QUICK RATIO = (CURRENT ASSETS-INVENTORY) / CURRENT LIABILITIES

# Profitability Ratios 
comp['roa'] = comp['ni'] / comp['at'] # ROA = NET INCOME / TOTAL ASSETS
comp['roe'] = comp['ni'] / comp['seq'] # ROE = NET INCOME / TOTAL SHAREHOLDER'S EQUITY
comp['gross_margin'] = (comp['sale'] - comp['cogs']) / comp['sale'] # GROSS MARGIN = (SALES - COGS)/ SALES
comp['operating_margin'] = comp['oibdp'] / comp['sale'] # OPERATING MARGIN = OPERATING INCOME / SALES

# Efficiency Ratios 
comp['lag_invt'] = comp.groupby('tic')['invt'].shift() # Compute Lag Inventory 
comp['avg_invt'] = (comp['invt'] + comp['lag_invt']) / 2 # Compute Average Inventory

comp['days_in_inventory'] = comp['avg_invt'] / (comp['cogs']/360) # DAYS IN INVENTORY = AVERAGE INVENTORY / (COGS/360)
comp['inventory_turns'] = 360 / comp['days_in_inventory'] # INVENTORY TURNS = 360 / DAYS IN INVENTORY

comp['days_sales_outstanding'] = comp['rect'] / (comp['sale']/360) # DAYS SALES OUTSTANDING =  ACCOUNTS RECEIVABLE / (SALES/360)
comp['days_payable_outstanding'] = comp['ap'] / (comp['cogs']/360) # DAYS PAYABLE OUTSTANDING = ACCOUNTS PAYABLE / (COSGS/360)

comp['ppe_turnover'] = comp['sale'] / comp['ppent'] # PPE TURNOVER = SALES / PPE 
comp['asset_turnover'] = comp['sale'] / comp['at'] # ASSET TURNOVER = SALES / TOTAL ASSETS


In [7]:
comp['cash_conversion_cycle'] = calculate_ccc(dio=comp['days_in_inventory'], dso=comp['days_sales_outstanding'], dpo=comp['days_payable_outstanding'])

In [8]:
# View last available values
comp.sort_values('datadate').groupby('tic').tail(1)

,gvkey,tic,conm,datadate,lt,at,lct,act,seq,dltt,...,operating_margin,lag_invt,avg_invt,days_in_inventory,inventory_turns,days_sales_outstanding,days_payable_outstanding,ppe_turnover,asset_turnover,cash_conversion_cycle
13,001690,AAPL,APPLE INC,2023-09-30,290437.0,352583.0,145308.0,143566.0,62146.0,106548.0,...,0.320391,4946.0,5638.5,9.871084,36.470161,57.280092,109.61043,7.04879,1.087077,-42.459255
